<a href="https://colab.research.google.com/github/roberthsheng/100pgml/blob/main/transforms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import torch
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

ds = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor(),
    target_transform=Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
)

In [14]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

In [7]:
device = (
    'cuda' if torch.cuda.is_available()
    else 'mps' if torch.backends.mps.is_available()
    else 'cpu'
)

print (f'Using {device} device')

Using cpu device


In [8]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
      nn.Linear(28*28, 512),
      nn.ReLU(),
      nn.Linear(512, 512),
      nn.ReLU(),
      nn.Linear(512, 10)
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [10]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f'Predicted class: {y_pred}')

Predicted class: tensor([8])


In [19]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


In [21]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [23]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [24]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.4474, -0.3281,  0.4162, -0.1573,  0.0759, -0.5605,  0.4134,  0.0032,
         -0.1557, -0.0026, -0.0855, -0.5589, -0.0394,  0.2897, -0.3407, -0.2325,
          0.3583, -0.2808,  0.0190, -0.3384],
        [-0.1897, -0.1417,  0.2484, -0.2453, -0.0728, -0.4681,  0.5799,  0.0904,
         -0.2801, -0.2072,  0.0939, -0.3261, -0.9392,  0.2739, -0.4114, -0.1318,
          0.2997,  0.0923, -0.2121, -0.2239],
        [-0.2269,  0.0692,  0.3947, -0.3468, -0.0067, -0.5890,  0.3243,  0.0065,
         -0.5286, -0.0523, -0.0135, -0.5765, -0.0968, -0.0682, -0.5213, -0.3590,
          0.1792, -0.0449, -0.0200, -0.5088]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.4162, 0.0000, 0.0759, 0.0000, 0.4134, 0.0032, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.2897, 0.0000, 0.0000, 0.3583, 0.0000,
         0.0190, 0.0000],
        [0.0000, 0.0000, 0.2484, 0.0000, 0.0000, 0.0000, 0.5799, 0.0904, 0.0000,
         0.0000, 0.0939, 0.0000, 0.0000, 0.2739, 0.00

In [25]:
seq_modules = nn.Sequential(
  flatten,
  layer1,
  nn.ReLU(),
  nn.Linear(20, 10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

In [26]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [27]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
  print(f"Layer: {name} | Size: {param.size()} | Values: {param[:2]}\n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[-0.0211, -0.0294, -0.0217,  ...,  0.0172,  0.0214, -0.0070],
        [-0.0169,  0.0335, -0.0330,  ..., -0.0112,  0.0006,  0.0150]],
       grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([-0.0295,  0.0202], grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[ 0.0371,  0.0284,  0.0409,  ...,  0.0308,  0.0343,  0.0325],
        [ 0.0259, -0.0275, -0.0166,  ..., -0.0322, -0.0035, -0.0322]],
       grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.2.bias | Size: 